# **Sprawozdanie 1 - Big data: przetwarzanie i analiza dużych zbiorów danych**
### Imię i nazwisko: Mateusz Boguszewski
### Data: 11.05.2024




Zostalo zastosowane podejscie wybrania waznych cech przed podziałem danych na treningowe oraz testowe. Powodem byl problem z zamiana danych tekstowych na danych testowych poniewaz zawieraly one dodatkowe pozycje, ktore nie trafily do danych treningowych. Nie mialem pomyslu jak to rozwiazac wiec zastosowalem takie podejscie.






## **I. Listing, kod modułu 1**

Dane byly trudne do polaczenia poniewaz dane kliniczne zawieraly kolumne z ID pacjenta a dane genetyczne mialy x kolumn w ktorych nazwach bylo ID pacjenta, dane mialy tez kilka wierszy co przy standardowym podejsciu do polaczenia ich dalo olbrzymia ramke danych z kilkukrotnie powtarzajcym sie pacjentem. Po polaczeniu danych uzupelniam brakujace wartosci, zamieniam wartosci tekstowe na numeryczne, normalizuje dane, pozbywam sie cech silnie skorelowanych ze zmienna decyzyjna a na koniec za pomoca RandomForest wybieram 100 najwazniejszych cech i zostawiam w danych tylko je.

In [ ]:
def modul1(kliniczne, genetyczne):

    clinical_data = pd.read_csv(kliniczne, comment='#', sep='\t')
    genetic_data = pd.read_csv(genetyczne, sep='\t')

    # Transpozycja danych
    genetic_data_transposed = genetic_data.transpose()
    new_columns = [f"{gene}_{id}" for gene, id in zip(genetic_data_transposed.iloc[0], genetic_data_transposed.iloc[1])]
    genetic_data_transposed.columns = new_columns

    # Usun te dwa pierwsze wiersze, są danymi kolumn
    genetic_data_transposed = genetic_data_transposed.drop(genetic_data_transposed.index[:2])

    # Usuniecie -1 z id pacjenta
    genetic_data_transposed.index = genetic_data_transposed.index.str.replace('-1', '')
    # print(genetic_data_transposed.head(50))

    # Polaczenie danych po ID pacjentów
    merged_data = pd.merge(clinical_data, genetic_data_transposed, left_on='PATIENT_ID', right_index=True)

    data = merged_data
    data['VITAL_STATUS'] = data['VITAL_STATUS'].map({'Alive': 1, 'Dead': 0})
    data = data.drop(['OTHER_PATIENT_ID'], axis=1)
    data.dropna(axis=1, how='all', inplace=True)

    # Imputacja brakujących danych

    imputer = SimpleImputer(strategy='mean')
    numeric_columns = data.select_dtypes(include=[np.number]).columns
    data[numeric_columns] = imputer.fit_transform(data[numeric_columns])

    categorical_columns = data.select_dtypes(include=[object]).columns

    # Imputacja najczęściej występującą wartością dla kolumn kategorycznych
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    data[categorical_columns] = categorical_imputer.fit_transform(data[categorical_columns])

    label_encoders = {}
    for column in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))
        label_encoders[column] = le

    # Wybieramy kolumny do normalizacji (wszystkie poza 'VITAL_STATUS')
    features = data.columns

    # Tworzymy instancję MinMaxScaler
    scaler = MinMaxScaler()

    # Normalizujemy dane
    data[features] = scaler.fit_transform(data[features])

    correlation_matrix = data.corr()

    # Wyciągnięcie korelacji względem 'VITAL_STATUS'
    correlation_with_target = correlation_matrix['VITAL_STATUS'].abs()

    # Wybór kolumn do usunięcia (próg korelacji ustawiony na 0.7)
    high_correlation_features = correlation_with_target[correlation_with_target > 0.7].index.tolist()
    high_correlation_features.remove('VITAL_STATUS')  # Usunięcie kolumny 'VITAL_STATUS' z listy

    # Usunięcie wybranych kolumn z DataFrame
    data = data.drop(columns=high_correlation_features)
    features = data.columns.difference(['VITAL_STATUS'])

    # Ponowna normalizacja danych
    data[features] = scaler.fit_transform(data[features])

    # Trenowanie lasu losowego dla selekcji cech
    rf = RandomForestClassifier(n_estimators=100, random_state=0)
    rf.fit(data.drop('VITAL_STATUS', axis=1), data['VITAL_STATUS'])

    # Wybór 100 najważniejszych cech
    important_features = pd.Series(rf.feature_importances_, index=data.columns[1:]).nlargest(100).index
    data = data[important_features.tolist()]

    return data

## **II. Listing, kod modułu 2**

In [ ]:
def modul2(data):
    # Podział danych na treningowe i testowe (przyjmujemy 70% trening, 30% test)
    train_data, test_data = train_test_split(data, test_size=0.3, random_state=0)

    # Trenowanie modelu SVM
    svm_model = SVC(kernel='rbf')
    svm_model.fit(train_data.drop('VITAL_STATUS', axis=1), train_data['VITAL_STATUS'])

    predictions = svm_model.predict(test_data.drop('VITAL_STATUS', axis=1))
    auc_score = roc_auc_score(test_data['VITAL_STATUS'], predictions)
    accuracy = accuracy_score(test_data['VITAL_STATUS'], predictions)
    cm = confusion_matrix(test_data['VITAL_STATUS'], predictions)
    print(cm)
    print(f"AUC Score: {auc_score}")
    print(f"Accuracy: {accuracy}")
    modul3(svm_model, predictions, test_data)

def data_abc(data, znacznik):

    predictors = data.columns.difference(['VITAL_STATUS'])

    if znacznik == 'A' or znacznik == 'a':
        DATA_A = data.copy()
        # Odejmowanie losowych 2% wartości początkowej każdego predyktora
        noise = np.random.rand(*DATA_A[predictors].shape) * 0.02
        DATA_A[predictors] -= DATA_A[predictors] * noise
        return modul2(DATA_A)

    elif znacznik == 'B' or znacznik == 'b':
        DATA_B = data.copy()
        # Dodawanie losowych 3% wartości początkowej każdego predyktora
        noise = np.random.rand(*DATA_B[predictors].shape) * 0.03
        DATA_B[predictors] += DATA_B[predictors] * noise

        # Usuwanie losowo 5% wierszy
        rows_to_drop = np.random.choice(DATA_B.index, size=int(len(DATA_B) * 0.05), replace=False)
        DATA_B = DATA_B.drop(rows_to_drop)
        return modul2(DATA_B)

    elif znacznik == 'C' or znacznik == 'c':
        DATA_C = data.copy()
        # Dodawanie losowych 3% wartości początkowej każdego predyktora
        noise = np.random.rand(*DATA_C[predictors].shape) * 0.03
        DATA_C[predictors] += DATA_C[predictors] * noise

        # Usuwanie losowo 20% kolumn predyktorów
        columns_to_drop = np.random.choice(predictors, size=int(len(predictors) * 0.2), replace=False)
        DATA_C = DATA_C.drop(columns=columns_to_drop)
        return modul2(DATA_C)

## **III. Listing, kod modułu 3**

Wyniki zostaly zwizualizowane przy pomocy macierzy pomylek oraz krzywej ROC

In [ ]:

def modul3(svm_model, predictions, test_data):
    # Macierz pomyłek
    cm = confusion_matrix(test_data['VITAL_STATUS'], predictions)

    # Wizualizacja macierzy pomyłek
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=['Dead', 'Alive'], yticklabels=['Dead', 'Alive'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    # Obliczanie wartości dla krzywej ROC
    fpr, tpr, thresholds = roc_curve(test_data['VITAL_STATUS'],
                                     svm_model.decision_function(test_data.drop('VITAL_STATUS', axis=1)))
    roc_auc = auc(fpr, tpr)

    # Rysowanie krzywej ROC
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()



## **IV. Listing, kod modułu 4**

### ***Opis:*** Poniżej przedstawiono etapy przykładowego procesu przetwarzania surowych danych, po połączeniu ramek danych zostały usunięte puste kolumny, w puste miejsca zostały dodane dane w zależności od typu danych w kolumnie: w numeryczne średnią wartość z danej kolumny i w tekstowych najczęściej występująca wartość. Dane zostały przekonwertowane do wartości numerycznych, a następnie wyskalowane. Następnie wykonałem badanie zależności cech do naszej kolumny decyzyjnej, te, które były wysoko zależne zostały usunięte z ramki. Ostatnim krokiem było stworzenie modelu lasów losowych do detekcji najważniejszych cech. Wybrałem 100 takich i zostawiłem tylko je w ramce. Po takim przekształceniu danych można je teraz podzielić na treningowe i testowe i śmiało budować model np. SVM do klasyfikacji osób chorych.

### **Import bibliotek**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt

###  **Odczyt danych**



In [ ]:
clinical_data = pd.read_csv('data_clinical_patient.txt', comment='#', sep='\t')
genetic_data = pd.read_csv('data_genetic.txt', sep='\t')

###  **Integracja danych**


In [ ]:
# Transpozycja danych
genetic_data_transposed = genetic_data.transpose()
new_columns = [f"{gene}_{id}" for gene, id in zip(genetic_data_transposed.iloc[0], genetic_data_transposed.iloc[1])]
genetic_data_transposed.columns = new_columns

# Usun te dwa pierwsze wiersze, są danymi kolumn
genetic_data_transposed = genetic_data_transposed.drop(genetic_data_transposed.index[:2])

# Usuniecie -1 z id pacjenta
genetic_data_transposed.index = genetic_data_transposed.index.str.replace('-1', '')
#print(genetic_data_transposed.head(50))

# Polaczenie danych po ID pacjentów
merged_data = pd.merge(clinical_data, genetic_data_transposed, left_on='PATIENT_ID', right_index=True)

data = merged_data

###  **Czyszczenie danych**

In [ ]:
data = data.drop(['OTHER_PATIENT_ID'], axis=1)
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column].astype(str))
    label_encoders[column] = le

###  **Imputacja danych**


In [ ]:
# Imputacja brakujących danych

imputer = SimpleImputer(strategy='mean')
numeric_columns = data.select_dtypes(include=[np.number]).columns
data[numeric_columns] = imputer.fit_transform(data[numeric_columns])

categorical_columns = data.select_dtypes(include=[object]).columns

# Imputacja najczęściej występującą wartością dla kolumn kategorycznych
categorical_imputer = SimpleImputer(strategy='most_frequent')
data[categorical_columns] = categorical_imputer.fit_transform(data[categorical_columns])
data.dropna(axis=1, how='all', inplace=True)



###  **Skalowanie**

In [ ]:
# Wybieramy kolumny do normalizacji (wszystkie poza 'VITAL_STATUS')
features = data.columns

# Tworzymy instancję MinMaxScaler
scaler = MinMaxScaler()

# Normalizujemy dane
data[features] = scaler.fit_transform(data[features])

###  **Badanie korelacji**

In [ ]:
correlation_matrix = data.corr()

# Wyciągnięcie korelacji względem 'VITAL_STATUS'
correlation_with_target = correlation_matrix['VITAL_STATUS'].abs()

# Wybór kolumn do usunięcia (próg korelacji ustawiony na 0.7)
high_correlation_features = correlation_with_target[correlation_with_target > 0.7].index.tolist()
high_correlation_features.remove('VITAL_STATUS')  # Usunięcie kolumny 'VITAL_STATUS' z listy

# Usunięcie wybranych kolumn z DataFrame
data = data.drop(columns=high_correlation_features)
features = data.columns.difference(['VITAL_STATUS'])

#Ponowna normalizacja danych
data[features] = scaler.fit_transform(data[features])

###  **Wybór 100 najlepszych cech**

In [ ]:
# Trenowanie lasu losowego dla selekcji cech
rf = RandomForestClassifier(n_estimators=100, random_state=0)
rf.fit(data.drop('VITAL_STATUS', axis=1), data['VITAL_STATUS'])

# Wybór 100 najważniejszych cech
important_features = pd.Series(rf.feature_importances_, index=data.columns[1:]).nlargest(100).index
data = data[important_features.tolist()]